In [ ]:
from pathlib import Path
import matplotlib as mpl
from flow_analysis_comps.data_structs.kymographs import kymoExtractConfig

import matplotlib.pyplot as plt
import colorcet  # noqa: F401
from flow_analysis_comps.processing.kymographing.kymographer import KymographCollection

plt.close("all")

%matplotlib widget

%load_ext autoreload
%autoreload 2

In [ ]:
from flow_analysis_comps.data_structs.kymographs import graphExtractConfig
from flow_analysis_comps.processing.graph_extraction.graph_extract import VideoGraphExtractor


video_root_folder = Path(
    r"/Users/simonvanstaalduine/AMOLF-SHIMIZU Dropbox/Simon van Staalduine/034"
)
info_file_address = Path(video_root_folder / "videoInfo.txt")
graph_data = VideoGraphExtractor(video_root_folder, graphExtractConfig())
kymograph_obj = KymographCollection(video_root_folder, kymoExtractConfig())


In [ ]:
kymographs = kymograph_obj.kymographs

print(kymographs)
for key, kymo in kymographs.items():
    plt.figure()
    plt.imshow(kymo, cmap="gray", origin="lower")
    plt.title(key)
    plt.colorbar()
    plt.show()

In [ ]:
from flow_analysis_comps.data_structs.kymographs import kymoDeltas
import numpy as np


video_deltas = kymoDeltas(
    delta_x=video_operator.space_pixel_size, delta_t=video_operator.time_pixel_size
)

# video_deltas = videoDeltas(
#     delta_t=1,
#     delta_x=1
# )

OSFilter = orientationSpaceManager(
    0.15,
    freq_width=0.2,
    K=16,
    x_spacing=video_deltas.delta_x,
    y_spacing=video_deltas.delta_t,
)

new_order = 8
speed_limit = 10

for key, kymo in kymographs.items():
    analyser = kymoAnalyser(
        kymo, video_deltas=video_deltas, speed_threshold=speed_limit
    )
    analyser.plot_kymo_fields()
    fig, ax = analyser.plot_summary()
    fig.savefig(video_root_folder / f"{key}_summary.png")
    image_l, image_r = analyser.kymograph_decomposed_directions
    invert_im = video_operator.video_info.mode == "brightfield"
    # _, _, histo_l = OSFilter.demo_image(
    #     image_l,
    #     video_operator.space_pixel_size,
    #     video_operator.time_pixel_size,
    #     order=new_order,
    #     thresh_method="triangle",
    #     invert=invert_im,
    #     histo_thresh=1.2,
    #     speed_extent=speed_limit,
    # )
    # _, _, histo_r = OSFilter.demo_image(
    #     image_r,
    #     video_operator.space_pixel_size,
    #     video_operator.time_pixel_size,
    #     order=new_order,
    #     thresh_method="triangle",
    #     invert=invert_im,
    #     histo_thresh=1.2,
    #     speed_extent=speed_limit,
    # )
    _, _, histo_b = OSFilter.demo_image(
        kymo,
        video_deltas.delta_x,
        video_deltas.delta_t,
        order=new_order,
        thresh_method="triangle",
        invert=invert_im,
        histo_thresh=0.7,
        speed_extent=speed_limit,
    )

In [ ]:
fig, ax = plt.subplots()
histo_l_n = histo_l / np.max(histo_l)
histo_r_n = histo_r / np.max(histo_r)
ax.imshow(
    (histo_l_n + histo_r_n).T,
    cmap="cet_CET_L16",
    extent=(0, len(histo_l) * video_deltas.delta_t, -speed_limit, speed_limit),
    aspect="auto",
)